In [ ]:
%load_ext autoreload
%autoreload
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath
import functools as ft

import sns_system, plotting_results
import distributed_sns
SS = distributed_sns.SimulationSet
ASS = distributed_sns.AggregatesSimulationSet

constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

In [ ]:
syst_pars = {'Ll' :  500,
               'Lr' : 500,
               'Lm' : 500,
               'Ly' : 25,
               'a' :  25,
            'mu_from_bottom_of_spin_orbit_bands': True}


In [ ]:
params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.0)
params = dict(**constants,
              **params_raw)

In [ ]:
def _params_Delta(syst_pars, params, Delta):
    for key in ['Delta_left', 'Delta_right']:
        if key in params:
            params[key] = Delta
        if key in syst_pars:
            syst_pars[key] = Delta
    return Delta

Deltas = np.linspace(0.1, 0.3, 3)

params_Delta = [ft.partial(_params_Delta,
                           Delta=_Delta
                           ) for _Delta in Deltas]

In [ ]:
def _params_alpha(syst_pars, params, alpha):
    for key in ['alpha_middle', 'alpha_left', 'alpha_right']:
        if key in params:
            params[key] = alpha
        if key in syst_pars:
            syst_pars[key] = alpha
    return alpha

alphas = np.linspace(30, 60, 3)

params_alpha = [ft.partial(_params_alpha,
                           alpha=_alpha
                           ) for _alpha in alphas]

In [ ]:
def _params_mu_ref(syst_pars, params, mu_ref):
    syst_pars['mu_from_bottom_of_spin_orbit_bands'] = mu_ref
    return mu_ref

mu_refs = [True, False]

params_mu_ref = [ft.partial(_params_mu_ref,
                           mu_ref=_mu_ref
                           ) for _mu_ref in mu_refs]

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,1.2]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = {
                       "pfaffian":{}}

# Make ASS

In [ ]:
%autoreload

In [ ]:
SOI_ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)

In [ ]:
SOI_ass.add_dimension('alpha', params_alpha)
SOI_ass.add_dimension('Delta', params_Delta)
SOI_ass.add_dimension('mu_ref', params_mu_ref)

# Build learner

In [ ]:
SOI_ass.make_balancing_learner(5000)

In [ ]:
bl = SOI_ass.get_balancing_learner()

# Runner Adaptive

In [ ]:
runner = adaptive.Runner(bl)
runner.live_info()

# Plot

In [ ]:
kdims, pd = SOI_ass.get_plot_dict(200)

In [ ]:
hv.HoloMap(pd, kdims=kdims)